In [12]:
# vectorized operations
import numpy as np

# dataframe manipulation
import pandas as pd

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# stats
import scipy.stats as stats

# stock datasets
from pydataset import data

# manipulation of time data
from datetime import date

# modeling preparation/data cleaning
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# filter out warnings
import warnings
warnings.filterwarnings('ignore')

# env (personal) function for sql db access
from env import get_db_url, user, password, host

# internal acquire method
import acquire

# show all columns for df.head()
pd.set_option("display.max_columns", None)
#pd.set_option("display.max_rows", None)

In [3]:
# 1. Read the data from attendance table and calculate attendance percentages. One half day is worth 50% of a full
# day, 10 tardies = to one absence
td_url = get_db_url('tidy_data')

In [27]:
query = 'SELECT * FROM attendance'
td_att = pd.read_sql(query, td_url)

In [28]:
td_att

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [29]:
td_att = td_att.melt(id_vars='Unnamed: 0')

In [30]:
td_att = td_att.rename(columns={'Unnamed: 0': 'Student', 'variable': 'date'})

In [31]:
td_att

,Student,date,value
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T
5,Jane,2018-01-02,P
6,Billy,2018-01-02,T
7,John,2018-01-02,T
8,Sally,2018-01-03,T
9,Jane,2018-01-03,T


In [32]:
td_att['value'] = td_att.value.map({'P': 1, 'T': .9, 'H': .5, 'A': 0})

In [37]:
attendance = td_att.groupby(by='Student').value.mean()

In [38]:
attendance

Student
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: value, dtype: float64

In [39]:
# 2. Read coffee_levels table, transform data so that each carafe is in it's own column
query = 'SELECT * FROM coffee_levels'
coffee = pd.read_sql(query, td_url)

In [41]:
coffee

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891
9,17,x,0.391560


In [44]:
coffee = coffee.pivot_table(index='hour', columns='coffee_carafe', values='coffee_amount')

In [45]:
coffee

coffee_carafe,x,y,z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947
13,0.310711,0.997464,0.398520
14,0.507288,0.058361,0.864464
15,0.215043,0.144644,0.436364
16,0.183891,0.544676,0.280621


In [51]:
coffee.reset_index(inplace=True)

In [55]:
coffee = coffee.drop(columns=['level_0', 'index'])

coffee_carafe,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947
5,13,0.310711,0.997464,0.398520
6,14,0.507288,0.058361,0.864464
7,15,0.215043,0.144644,0.436364
8,16,0.183891,0.544676,0.280621
9,17,0.391560,0.594126,0.436677


In [57]:
coffee.columns.name = ''

In [60]:
coffee

,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947
5,13,0.310711,0.997464,0.398520
6,14,0.507288,0.058361,0.864464
7,15,0.215043,0.144644,0.436364
8,16,0.183891,0.544676,0.280621
9,17,0.391560,0.594126,0.436677


In [61]:
# This is a useful shape if people want to look at the data for themselves, but probably not the best shape to use
# for modeling

In [62]:
# 3. Read the cake_recipes table
query = 'SELECT * FROM cake_recipes'
cakes = pd.read_sql(query, td_url)

In [63]:
cakes

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


In [65]:
cakes = cakes.melt(id_vars='recipe:position')

In [71]:
cakes['recipe'] = cakes['recipe:position'].str.split(':').str[0]

In [73]:
cakes['position'] = cakes['recipe:position'].str.split(':').str[1]

In [75]:
cakes = cakes.drop(columns='recipe:position')

In [79]:
# cakes.pivot_table(values='value', columns='variable', index=['recipe', 'position'])

variable               225        250        275        300
recipe position                                            
a      bottom    61.738655  53.912627  74.414730  98.786784
       top       51.709751  52.009735  68.576858  50.228470
b      bottom    57.095320  61.904369  61.196980  99.248541
       top       82.455004  95.224151  98.594881  58.169349
c      bottom    96.470207  52.001358  92.893227  65.473084
       top       71.306308  82.795477  92.098049  53.960273
d      bottom    52.799753  58.670419  51.747686  56.183110
       top       96.873178  76.101363  59.571620  50.971626

In [85]:
cakes.columns = ['oven_temp', 'score', 'recipe', 'oven_position']

In [87]:
cakes = cakes[['recipe', 'oven_position', 'oven_temp', 'score']]

In [91]:
cakes

,recipe,oven_position,oven_temp,score
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207
5,c,top,225,71.306308
6,d,bottom,225,52.799753
7,d,top,225,96.873178
8,a,bottom,250,53.912627
9,a,top,250,52.009735


In [99]:
# C. Which, on average, is the best recipe?
cakes.groupby(by='recipe').score.mean().idxmax()
# Recipe b performs the best overall

'b'

In [98]:
# D. Which oven temp., on average, produces the best results?
cakes.groupby(by='oven_temp').score.mean()
# 275 degrees performs the best on average

oven_temp
225    71.306022
250    66.577437
275    74.886754
300    66.627655
Name: score, dtype: float64

In [100]:
# E. What combination produces the best results?
winner = cakes[cakes.score.max() == cakes.score]
winner

,recipe,oven_position,oven_temp,score
26,b,bottom,300,99.248541


In [101]:
# 4. Bonus: Explore other tables in tidy_data and reshape them as necessary so they are in tidy format